  # Apply encoder to morphological profiles to get latent space representations :

# Setup

In [1]:
%load_ext autoreload  
%autoreload 2
from IPython.display import display, HTML, Image
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:98% !important; }</style>"))
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import sys
import random
from typing import List, Tuple
from types import SimpleNamespace
import pprint
import logging
from datetime import datetime
for p in ['../','../../..']:
    if p not in sys.path:
        print(f"insert {p}")
        sys.path.insert(0, p)
print(sys.path)

import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd

import scipy
import scipy.stats as sps
import sklearn.metrics as skm
from scipy.spatial.distance import pdist, squareform, euclidean

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt  # for making figures
from torchinfo import summary

torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=180, profile=None, sci_mode=None)
torch.manual_seed(42);   # seed rng for reproducibility
pp = pprint.PrettyPrinter(indent=4)
pd.options.display.width = 132
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')

os.environ["WANDB_NOTEBOOK_NAME"] = "AE-MAIN-SNNL.ipynb"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

torch.set_num_threads(4)  ## <--- limit to ~ 2 CPUs
torch.get_num_threads()

insert ../
insert ../../..
['../../..', '../', '/home/kevin/WSL-shared/cellpainting/cj-datasets/Notebooks-Pharmacophore_Classification', '/home/kevin/miniforge3/envs/cp311/lib/python311.zip', '/home/kevin/miniforge3/envs/cp311/lib/python3.11', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg']


4

In [3]:
# from KevinsRoutines.utils.utils_general import list_namespace, save_to_pickle, load_from_pickle, get_device
import KevinsRoutines.utils as myutils

# from utils.utils_ptsnnl import display_cellpainting_batch, get_device
from src.utils.utils_cellpainting import label_counts, balance_datasets,save_checkpoint, load_checkpoint

from src.utils.utils_notebooks import plot_cls_metrics, compute_classification_metrics, run_model_on_test_data,\
                                train, validation, accuracy_fn, fit, build_model, define_datasets

/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [4]:
timestamp = datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
logger = logging.getLogger(__name__)
logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
FORMAT = '%(asctime)s - %(name)s - %(levelname)s: - %(message)s'
logging.basicConfig(level="INFO", format= FORMAT)
logger.info(f" Excution started : {timestamp} ")
logger.info(f" Pytorch version  : {torch.__version__}")
logger.info(f" Scipy version    : {scipy.__version__}  \t\t Numpy version : {np.__version__}")
logger.info(f" Pandas version   : {pd.__version__}  ")

2024-12-09 18:43:06,898 - __main__ - INFO: -  Excution started : 2024_12_09_18:43:06 
2024-12-09 18:43:06,899 - __main__ - INFO: -  Pytorch version  : 2.2.0
2024-12-09 18:43:06,900 - __main__ - INFO: -  Scipy version    : 1.11.4  		 Numpy version : 1.26.2
2024-12-09 18:43:06,900 - __main__ - INFO: -  Pandas version   : 2.2.2  


In [48]:
from rdkit import RDConfig
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Pharm2D.SigFactory import SigFactory
fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
print(fdefName)
featFactory = ChemicalFeatures.BuildFeatureFactory(fdefName)
sigFactory = SigFactory(featFactory,minPointCount=2,maxPointCount=3)
sigFactory.SetBins([(0,3),(3,8)])
sigFactory.Init()
sigFactory.GetSigSize()
bit_id = 0
print(f" bit: {bit_id:4d}   description:  {sigFactory.GetBitDescription(bit_id)}")

# Phramacophore bits with a ROC_AUC pver .60
selected_bits_over60   = [23, 17, 53, 21, 509,  9, 52, 49, 163, 797,
                          39, 13, 267, 141, 285,  479, 677, 45, 127, 270,
                          767, 468, 271, 2, 165,  756, 27, 387, 26, 287,
                          390, 48, 231, 183, 383, 8]

# Phramacophore bits with a ROC_AUC between .55 and .60
selected_bits_55_to_60 = [158, 425, 108, 33, 148, 421, 606, 647, 86, 638,
                          433, 111, 94, 246, 91, 87, 430, 80, 110, 198,
                          639, 104, 35, 422, 426, 42, 0, 0, 5, 78,
                          19, 154, 370, 20, 210, 464, 470, 212, 646, 599,
                          247, 633, 437, 245, 595, 250, 30, 439, 371, 34,
                          375, 202, 142, 758, 643, 197, 598, 469, 199, 166,
                          465, 382, 427, 255, 374, 44, 43, 252, 179, 159,
                          31, 167, 254, 143, 438, 641, 162, 366, 379, 478,
                          645, 157, 57, 372, 434, 153, 752, 477, 140, 471,
                          763, 435, 251, 473, 253, 164, 679, 22, 475, 766,
                          378, 155, 152, 16, 759]

# Phramacophore bits with a ROC_AUC between .50 and .55
selected_bits_50_to_55 = [200, 205, 211, 428, 642, 103,  98, 215, 100, 636,
                          150, 149, 144, 102, 207, 151, 206,   6, 147,  79,
                          83,  95, 203, 607, 603, 213, 208, 107,  99,  32,
                          431, 204, 96,  214,   4,   1, 637, 146, 429, 156,
                          417, 18, 244,  423, 84]
print()
print(f" Num     Bit    ROC_AUC     Description")
print(f" ----    ----   --------    -------------")
for idx, bit_id in enumerate(sorted(selected_bits_over60),1):    
    if bit_id in selected_bits_over60: 
        prefix = " > 0.60  "
    elif bit_id in selected_bits_55_to_60:
        prefix = " > 0.55  "
    elif bit_id in selected_bits_50_to_55:
        prefix = " > 0.50  "
    else :
        prefix = "         "
    print(f" {idx: 4d}    {bit_id:4d}   {prefix:9s}    {sigFactory.GetBitDescription(bit_id)}")
 

/home/kevin/miniforge3/envs/cp311/share/RDKit/Data/BaseFeatures.fdef


1032

 bit:    0   description:  Acceptor Acceptor |0 0|0 0|

 Num     Bit    ROC_AUC     Description
 ----    ----   --------    -------------
    1       2    > 0.60      Acceptor Aromatic |0 0|0 0|
    2       8    > 0.60      Acceptor LumpedHydrophobe |0 0|0 0|
    3       9    > 0.60      Acceptor LumpedHydrophobe |0 1|1 0|
    4      13    > 0.60      Acceptor PosIonizable |0 1|1 0|
    5      17    > 0.60      Aromatic Aromatic |0 1|1 0|
    6      21    > 0.60      Aromatic Hydrophobe |0 1|1 0|
    7      23    > 0.60      Aromatic LumpedHydrophobe |0 1|1 0|
    8      26    > 0.60      Aromatic PosIonizable |0 0|0 0|
    9      27    > 0.60      Aromatic PosIonizable |0 1|1 0|
   10      39    > 0.60      Donor PosIonizable |0 1|1 0|
   11      45    > 0.60      Hydrophobe LumpedHydrophobe |0 1|1 0|
   12      48    > 0.60      Hydrophobe PosIonizable |0 0|0 0|
   13      49    > 0.60      Hydrophobe PosIonizable |0 1|1 0|
   14      52    > 0.60      LumpedHydrophobe LumpedHydropho

In [6]:
try:
    del model
except Exception as e:
    pass

In [7]:
myutils.set_device(1)
device  = myutils.get_device(verbose = True)
print(device)

 Switched to: "cuda:1"   Device Name: Quadro GV100                  


'cuda:1'

Dev Id   Device Name                    Total Memory                     InUse                            Free Memory 
   0     Quadro GV100                   34,069,872,640 B/ (31.73 GB)  	 325,058,560 B / (0.30 GB)  	 33,744,814,080 B / (31.43 GB)  
   1     Quadro GV100                   34,069,872,640 B/ (31.73 GB)  	 325,058,560 B / (0.30 GB)  	 33,744,814,080 B / (31.43 GB)   *** CURRENT DEVICE *** 
   2     NVIDIA TITAN Xp                12,774,539,264 B/ (11.90 GB)  	 157,417,472 B / (0.15 GB)  	 12,617,121,792 B / (11.75 GB)  

 Current CUDA Device is:  "cuda:1"  Device Name: Quadro GV100
cuda:1


# Args 

In [8]:
LATENT_DIM    = 250
COMPOUNDS_PER_BATCH = 600

# MODEL_TYPE = 'batch_norm'
# MODEL_TYPE = 'single_layer'
MODEL_TYPE = 'relu'
n_input    = LATENT_DIM  # the embedding dimensionality 
n_hidden_1 = 512  # the number of neurons in the hidden layer of the MLP
n_hidden_2 = 512  # the number of neurons in the hidden layer of the MLP
n_hidden_3 = 128
output     = 12

METADATA_COLS = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA', 'Metadata_Permiation']
# METADATA_COLS += [f'Feature_{x:03d}' for x in range(LATENT_DIM)]
input_cols = LATENT_DIM + len(METADATA_COLS)
print(len(METADATA_COLS))
print(input_cols)

INPUT_PATH = f"/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/"
PHARMACOPHORE_INPUT_PATH = f"/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/"
CKPT_PATH = "./saved_models/embedding_models"

11
261


In [9]:
# RUN_DATETIME = datetime.now().strftime('%Y%m%d_%H%M')
# RUN_DATETIME = '20240909_1801'
# RUN_DATETIME = '20240909_1800'
# RUN_DATETIME = '20240909_2100'
# RUN_DATETIME = '20240916_1830'
# RUN_DATETIME = '20240921_0700'
# RUN_DATETIME = '20240926_1900'
# RUN_DATETIME = '20240927_2345'
# RUN_DATETIME = '20240929_2015'
# RUN_DATETIME = '20240930_1930'
RUN_DATETIME = '20241209_1600'
print(RUN_DATETIME)

20241209_1600


In [10]:
# SNNL AUTOENCODERS 
AE_RUNMODE = "snnl"
# AE_DATETIME = "20240718_1956"
# AE_DATETIME = "20240906_2201"     # Autoencoder training - SNNL, CPB = 600, Latent 150, WD = 0.001, SNN Factor 3
AE_DATETIME = "20240917_2004"     # Autoencoder training - SNNL, CPB = 600, Latent 250, WD = 0.001, SNN Factor 3

## BASELINE AUTOENCODERS 
# AE_RUNMODE = 'base'
# AE_DATETIME = "20240917_2017"     # Autoencoder training - Baseline, CPB = 600, Latent 250, WD = 0.001 (SNN Factor 0)

# AE_CKPTTYPE = "BEST"
AE_CKPTTYPE = "LAST"

In [11]:
CKPT_FILE = f"NN_{AE_RUNMODE.lower()}_embd600_{LATENT_DIM}Ltnt_512_{AE_DATETIME}_{AE_CKPTTYPE}_{RUN_DATETIME}_ep_{{ep}}"
print(CKPT_FILE)

NN_snnl_embd600_250Ltnt_512_20240917_2004_LAST_20241209_1600_ep_{ep}


In [66]:
## total rows = 346,542
## Trn file sz: 312,000 
## Train      : 277,200    (312_000 - (21,600 + 12,600 + 600) = 277,200
## Validation :  21,600
## Test       :  12,600
## Leftover   :     600
cellpainting_args = {'compounds_per_batch': COMPOUNDS_PER_BATCH,
                     'train_start'        : 0,
                     'train_end'          : 277_200,
                     'val_start'          : 0,
                     'val_end'            : 21_600,
                     'test_start'         : 0,
                     'test_end'           : 12_600, 
                    }
pharmacophore_args = {'compounds_per_batch': COMPOUNDS_PER_BATCH,
                     'train_start'        : 0,
                     'train_end'          : 277_200,
                     'val_start'          : 0,
                     'val_end'            : 21_600,
                     'test_start'         : 0,
                     'test_end'           : 12_600, 
                     'return_bits' : sorted(selected_bits_over60)
                    }
cellpainting_args
pharmacophore_args
INPUT_PATH

{'compounds_per_batch': 600,
 'train_start': 0,
 'train_end': 277200,
 'val_start': 0,
 'val_end': 21600,
 'test_start': 0,
 'test_end': 12600}

{'compounds_per_batch': 600,
 'train_start': 0,
 'train_end': 277200,
 'val_start': 0,
 'val_end': 21600,
 'test_start': 0,
 'test_end': 12600,
 'return_bits': [2,
  8,
  9,
  13,
  17,
  21,
  23,
  26,
  27,
  39,
  45,
  48,
  49,
  52,
  53,
  127,
  141,
  163,
  165,
  183,
  231,
  267,
  270,
  271,
  285,
  287,
  383,
  387,
  390,
  468,
  479,
  509,
  677,
  756,
  767,
  797]}

'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/'

In [67]:
data_loader = define_datasets(cellpainting_args, AE_RUNMODE, AE_DATETIME, input_cols, AE_CKPTTYPE, INPUT_PATH)

2024-12-09 19:11:27,667 - src.utils.dataloader - INFO: -  Building CellPantingDataset for train
2024-12-09 19:11:27,668 - src.utils.dataloader - INFO: -  filename:  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_261_HashOrder_snnl_20240917_2004_LAST_training.csv
2024-12-09 19:11:27,670 - src.utils.dataloader - INFO: -  type    :  train
2024-12-09 19:11:27,671 - src.utils.dataloader - INFO: -  start   :  0
2024-12-09 19:11:27,671 - src.utils.dataloader - INFO: -  end     :  277200
2024-12-09 19:11:27,673 - src.utils.dataloader - INFO: -  numrows :  277200
2024-12-09 19:11:27,673 - src.utils.dataloader - INFO: -  names   :  None     usecols :  None
2024-12-09 19:11:27,675 - src.utils.dataloader - INFO: -  batch_size  :  1
2024-12-09 19:11:27,675 - src.utils.dataloader - INFO: -  sample_size :  3
2024-12-09 19:11:27,676 - src.utils.dataloader - INFO: -  compounds_per_batch :  600
2024-12-09 19:11:27,677 - src.utils.dataloader - INF

 TRAIN_INPUT:  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_261_HashOrder_snnl_20240917_2004_LAST_training.csv
 TEST_INPUT :  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_261_HashOrder_snnl_20240917_2004_LAST_test.csv
 ALL_INPUT  :  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_261_HashOrder_snnl_20240917_2004_LAST_validation.csv
 load {}
 Dataset size: 277200   rows per batch: 1800 
 Dataset size: 21600   rows per batch: 1800 
 Dataset size: 12600   rows per batch: 1800 


In [65]:
dd = np.array(pharmacophore_args['return_bits'])
dd

array([  2,   8,   9,  13,  17,  21,  23,  26,  27,  39,  45,  48,  49,  52,  53, 127, 141, 163, 165, 183, 231, 267, 270, 271, 285, 287, 383, 387,
       390, 468, 479, 509, 677, 756, 767, 797])

In [68]:
from src.utils.dataloader import InfiniteDataLoader, PharmacophoreDataset, pharmacophore_collate_fn
from functools import partial
def define_pharmacopohre_datasets(pharmacophore_args, input_path):

    num_cols = 1482
    TRAIN_INPUT_FILE = f"3sample_pharmacophores_{num_cols}_HashOrder_training.csv"
    TEST_INPUT_FILE  = f"3sample_pharmacophores_{num_cols}_HashOrder_test.csv"
    VAL_INPUT_FILE   = f"3sample_pharmacophores_{num_cols}_HashOrder_validation.csv"

    TRAIN_INPUT = os.path.join(input_path, TRAIN_INPUT_FILE)
    TEST_INPUT  = os.path.join(input_path, TEST_INPUT_FILE)
    VAL_INPUT   = os.path.join(input_path, VAL_INPUT_FILE)

    print(f" TRAIN_INPUT:  {TRAIN_INPUT}")
    print(f" TEST_INPUT :  {TEST_INPUT }")
    print(f" ALL_INPUT  :  {VAL_INPUT }")

    pharmacophore_args['training_path'] = TRAIN_INPUT
    pharmacophore_args['validation_path'] = VAL_INPUT
    pharmacophore_args['test_path'] = TEST_INPUT

    # Load Pharmacophore Dataset
    # data : keys to the dataset settings (and resulting keys in output dictionary)
    dataset = dict()
    data_loader = dict()

    print(f" load {dataset}")
    for datatype in ['train', 'val', 'test']:
        dataset[datatype] = PharmacophoreDataset(type = datatype, **pharmacophore_args)
        data_loader[datatype] = InfiniteDataLoader(dataset = dataset[datatype], batch_size=1, shuffle = False, num_workers = 0,
                                                   collate_fn = partial(pharmacophore_collate_fn, return_bits = dataset[datatype].return_bits) )

    return data_loader

In [69]:
pp_data_loader = define_pharmacopohre_datasets(pharmacophore_args, PHARMACOPHORE_INPUT_PATH)

2024-12-09 19:11:38,321 - src.utils.dataloader - INFO: -  Building Pharmacophore Dataset for train
2024-12-09 19:11:38,322 - src.utils.dataloader - INFO: -  filename:  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_pharmacophores_1482_HashOrder_training.csv
2024-12-09 19:11:38,324 - src.utils.dataloader - INFO: -  type    :  train
2024-12-09 19:11:38,325 - src.utils.dataloader - INFO: -  start   :  0
2024-12-09 19:11:38,326 - src.utils.dataloader - INFO: -  end     :  277200
2024-12-09 19:11:38,327 - src.utils.dataloader - INFO: -  numrows :  277200
2024-12-09 19:11:38,328 - src.utils.dataloader - INFO: -  names   :  None     usecols :  None
2024-12-09 19:11:38,329 - src.utils.dataloader - INFO: -  batch_size  :  1
2024-12-09 19:11:38,330 - src.utils.dataloader - INFO: -  sample_size :  3
2024-12-09 19:11:38,330 - src.utils.dataloader - INFO: -  compounds_per_batch :  600
2024-12-09 19:11:38,331 - src.utils.dataloader - INFO: -  rows per batch

 TRAIN_INPUT:  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_pharmacophores_1482_HashOrder_training.csv
 TEST_INPUT :  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_pharmacophores_1482_HashOrder_test.csv
 ALL_INPUT  :  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_pharmacophores_1482_HashOrder_validation.csv
 load {}
 Dataset size: 277200   rows per batch: 1800 
 Dataset size: 21600   rows per batch: 1800 
 Dataset size: 12600   rows per batch: 1800 


## Input Dataloader


In [73]:
from src.utils.utils_notebooks import display_pharmacophore_batch
# for dataset in ['train', 'val', 'test']:
for dataset in ['test']:
    for idx, batch in enumerate(pp_data_loader[dataset]):
        print(batch[0].shape[0], batch[1].sum())
        display_pharmacophore_batch(idx, batch)
        if idx == 1:
            break

 batch shape: 1   batch_numpy: (1800, 1035)
 type: <class 'pandas.core.frame.DataFrame'> shape: (1800, 1035)
[  2   8   9  13  17  21  23  26  27  39  45  48  49  52  53 127 141 163 165 183 231 267 270 271 285 287 383 387 390 468 479 509 677 756 767 797]
[[0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0]
 [0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0]
 [0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0]
 [0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0]
 [0 1 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 1 0 1 0 

In [76]:
from src.utils.utils_notebooks import display_cellpainting_batch
# for dataset in ['train', 'val', 'test']:
for dataset in ['test']:
    for idx, batch in enumerate(data_loader[dataset]):
        print(batch[0].shape[0], batch[1].sum())
        display_cellpainting_batch(idx, batch)
        if idx == 1:
            break

1800 tensor(204.)
---------------------------------------------------------------------------------------------------------------------------------------
  Batch Id: 0   <class 'tuple'>  Rows returned 1800 features: torch.Size([1800, 250])  
+-----+------------------------------------------+----------------+--------------------------+------------------------------+-----+-----+--------------------------------------------------------+
| idx |   batch[2]                               |    batch[3]    |      batch[2]            |          batch[5]            | [1] | [1] |     batch[0]                                           | 
|     | SRCE      BATCH     PLATE     WELL       |   COMPOUND_ID  |       CMPHASH / BIN      |  TPSA / Ln(TPSA) / Log(TPSA) | LBL |LBL2 |     FEATURES                                           | 
+-----+------------------------------------------+----------------+--------------------------+------------------------------+-----+-----+----------------------------------

In [42]:
# # -----------------------------------------
# #  Count pos/neg labels in each dataset
# # -----------------------------------------
for datatype in ['train', 'val', 'test']:
    MINIBATCH_SIZE = data_loader[datatype].dataset.sample_size * data_loader[datatype].dataset.compounds_per_batch
    print(f" {datatype.capitalize()} Minibatch size : {MINIBATCH_SIZE} \n") 
# for datatype in ['val', 'test']:
    minibatches = len(data_loader[datatype]) // MINIBATCH_SIZE
    ttl_rows = 0
    ttl_pos_labels = 0
    with tqdm.tqdm(enumerate(data_loader[datatype]), initial=0, total = minibatches, position=0, file=sys.stdout,
                   leave= False, desc=f" Count labels ") as t_warmup:
        for batch_count, (batch_features, batch_labels, plates, compounds, cmphash, tpsa, orig_labels) in t_warmup:
            ttl_rows += batch_labels.shape[0]
            ttl_pos_labels += batch_labels.sum()
    ttl_neg_labels = ttl_rows - ttl_pos_labels
    ttl = f"\n Dataset: {datatype} -  len of {datatype} data loader: {len(data_loader[datatype])}   number of batches: {minibatches}"
    print(ttl)
    print('-'*len(ttl))
    print(f" total rows     : {ttl_rows:7d}")
    print(f" total pos rows : {ttl_pos_labels:7.0f} - {ttl_pos_labels*100.0/ttl_rows:5.2f}%")
    print(f" total neg rows : {ttl_neg_labels:7.0f} - {ttl_neg_labels*100.0/ttl_rows:5.2f}%")
    print()

 Train Minibatch size : 1800 

                                                                                                                                                                                                                                        
 Dataset: train -  len of train data loader: 277200   number of batches: 154
-----------------------------------------------------------------------------
 total rows     :  277200
 total pos rows :   33120 - 11.95%
 total neg rows :  244080 - 88.05%

 Val Minibatch size : 1800 

                                                                                                                                                                                                                                        
 Dataset: val -  len of val data loader: 21600   number of batches: 12
-----------------------------------------------------------------------
 total rows     :   21600
 total pos rows :    2532 - 11.72%
 total neg rows : 

# Define Neural Net Model 

- **4 layer model :**

    Input --> Hidden1 --> (BN/NL) ---> Hidden2 ---> (BN/NL) ---> Hidden3 --->  (BN/NL) ---> 1
   
    -  **20240909_1800** : Run on 4 FC layers model, model configuration UNKNOWN
    -  **20240909_1801** : Run on 4 FC layers model, Relu non linearities (NO Batch Norm)
    -  **20240909_2100** : Run on 4 FC layers model, with BATCH NORM / Tanh 256,256,128

    -  **20240930_1930** : Run on 4 FC layers model, with BATCH NORM/ Tanh 512,512,128
      
 - **Single Hidden Layer - 256**

   Input --> Hidden1 --> (Tanh) --->  1
    -  **20240916_1830** : Run on 1 FC layers model, Input --> 256 --> Tanh --> 1 ,  Read from 20240906_2201 (SNNL - CPB 600, LAT 150, SNN Factor 3)
    -  **20240926_1900** : Run on 1 FC layers model, Input --> 256 --> Tanh --> 1 ,  Read from 20240917_2017 (BASELINE - CPB 600, LAT 250, SNN Factor 0)
    -  **20240926_1930** : Run on 1 FC layers model, Input --> 256 --> Tanh --> 1 ,  Read from 20240917_2004 (SNNL - CPB 600, LAT 250, SNN Factor 3)
    -  **20240926_2000** : Run on 1 FC layers model, Input --> 256 --> Tanh --> 1 ,  Read from 20240924_0146 (SNNL - CPB 600, LAT 250, SNN Factor 30)
<br>

 - **Single Hidden Layer - 256**

    -  **20240921_0700** : Run on 1 FC layers model (includes final layer), Input --> 512 --> Tanh --> 1 ,  Read from 20240906_2201 (SNNL - CPB 600, LAT 150, SNN Factor 3)    


In [15]:
model = build_model(MODEL_TYPE, input = n_input, hidden_1 = n_hidden_1, hidden_2 = n_hidden_2, hidden_3=n_hidden_3, device = device)

In [19]:
metrics = { 'loss_trn' : [], 'acc_trn' : [], 'loss_val' : [], 'acc_val' : []}

start_epoch, end_epoch = 0,0
init_LR = 1.0e-3
# curr_LR = init_LR

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=init_LR)

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.3 , patience=20, cooldown=10,)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = step_size, gamma=0.1, last_epoch =-1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.5, threshold=1.0e-06, patience=50, cooldown=10,)

### Read checkpoint

In [20]:
# loaded_epoch
# optimizer.state_dict()
# scheduler.state_dict()

In [21]:
# model, optimizer, scheudler, end_epoch = load_checkpoint(model, optimizer, scheduler, checkpoint_file.format(ep=100), ckpt_path = CKPT_PATH)
# model = model.to(device)

In [22]:
# end_epoch
# optimizer.state_dict()
# scheduler.state_dict()

# Run Training

In [23]:
# start_epoch = 0
# start_epoch = loaded_epoch
start_epoch = end_epoch
end_epoch += 600
# start_epoch, end_epoch = 0,100
print(start_epoch, end_epoch)
_ = model.train()

0 600


In [24]:

_ = fit(model, optimizer, scheduler, data_loader, metrics, start_epoch, end_epoch, device, CKPT_FILE, CKPT_PATH )


 19:42:58 | Ep:   1/ 600 | Trn loss:  0.398827 - Acc: 86.3727 | Val loss:  0.351597 - Acc: 88.2407 | last_lr: 1.00000e-03  bad_ep: 0  cdwn: 0                              
 19:43:21 | Ep:   2/ 600 | Trn loss:  0.358226 - Acc: 87.9942 | Val loss:  0.352385 - Acc: 88.2454 | last_lr: 1.00000e-03  bad_ep: 1  cdwn: 0                              
 19:43:45 | Ep:   3/ 600 | Trn loss:  0.355861 - Acc: 88.0249 | Val loss:  0.352876 - Acc: 88.2407 | last_lr: 1.00000e-03  bad_ep: 2  cdwn: 0                              
 19:44:08 | Ep:   4/ 600 | Trn loss:  0.354865 - Acc: 88.0317 | Val loss:  0.353677 - Acc: 88.2176 | last_lr: 1.00000e-03  bad_ep: 3  cdwn: 0                              
 19:44:31 | Ep:   5/ 600 | Trn loss:  0.354195 - Acc: 88.0426 | Val loss:  0.353060 - Acc: 88.2407 | last_lr: 1.00000e-03  bad_ep: 4  cdwn: 0                              
 19:44:54 | Ep:   6/ 600 | Trn loss:  0.353654 - Acc: 88.0480 | Val loss:  0.351972 - Acc: 88.2778 | last_lr: 1.00000e-03  bad_ep: 5  cdwn: 

2024-09-30 20:22:18,907 - utils.utils_cellpainting - INFO: -  Model exported to NN_snnl_embd600_250Ltnt_512_20240917_2004_LAST_20240930_1930_ep_100.pt - epoch: 100


 20:22:18 | Ep: 100/ 600 | Trn loss:  0.321803 - Acc: 88.8232 | Val loss:  0.436422 - Acc: 87.6250 | last_lr: 5.00000e-04  bad_ep: 1  cdwn: 0 
 20:22:42 | Ep: 101/ 600 | Trn loss:  0.321032 - Acc: 88.8405 | Val loss:  0.439582 - Acc: 87.5231 | last_lr: 5.00000e-04  bad_ep: 2  cdwn: 0                              
 20:23:06 | Ep: 102/ 600 | Trn loss:  0.320279 - Acc: 88.8622 | Val loss:  0.439251 - Acc: 87.4769 | last_lr: 5.00000e-04  bad_ep: 3  cdwn: 0                              
 20:23:30 | Ep: 103/ 600 | Trn loss:  0.319522 - Acc: 88.8900 | Val loss:  0.439580 - Acc: 87.4630 | last_lr: 5.00000e-04  bad_ep: 4  cdwn: 0                              
 20:23:54 | Ep: 104/ 600 | Trn loss:  0.318787 - Acc: 88.9098 | Val loss:  0.437508 - Acc: 87.6065 | last_lr: 5.00000e-04  bad_ep: 5  cdwn: 0                              
 20:24:18 | Ep: 105/ 600 | Trn loss:  0.318053 - Acc: 88.9325 | Val loss:  0.435091 - Acc: 87.6806 | last_lr: 5.00000e-04  bad_ep: 6  cdwn: 0                            

2024-09-30 21:02:57,613 - utils.utils_cellpainting - INFO: -  Model exported to NN_snnl_embd600_250Ltnt_512_20240917_2004_LAST_20240930_1930_ep_200.pt - epoch: 200


 21:02:57 | Ep: 200/ 600 | Trn loss:  0.263652 - Acc: 90.5047 | Val loss:  0.423765 - Acc: 87.7917 | last_lr: 2.50000e-04  bad_ep: 40  cdwn: 0 
 21:03:22 | Ep: 201/ 600 | Trn loss:  0.263151 - Acc: 90.5263 | Val loss:  0.425724 - Acc: 87.7870 | last_lr: 2.50000e-04  bad_ep: 41  cdwn: 0                             
 21:03:46 | Ep: 202/ 600 | Trn loss:  0.262663 - Acc: 90.5411 | Val loss:  0.427152 - Acc: 87.7824 | last_lr: 2.50000e-04  bad_ep: 42  cdwn: 0                             
 21:04:11 | Ep: 203/ 600 | Trn loss:  0.262187 - Acc: 90.5588 | Val loss:  0.428904 - Acc: 87.7778 | last_lr: 2.50000e-04  bad_ep: 43  cdwn: 0                             
 21:04:35 | Ep: 204/ 600 | Trn loss:  0.261730 - Acc: 90.5758 | Val loss:  0.430261 - Acc: 87.7778 | last_lr: 2.50000e-04  bad_ep: 44  cdwn: 0                             
 21:04:59 | Ep: 205/ 600 | Trn loss:  0.261290 - Acc: 90.5848 | Val loss:  0.432208 - Acc: 87.7870 | last_lr: 2.50000e-04  bad_ep: 45  cdwn: 0                          

...

 22:24:38 | Ep: 400/ 600 | Trn loss:  0.230068 - Acc: 91.6219 | Val loss:  0.433319 - Acc: 87.2824 | last_lr: 1.56250e-05  bad_ep: 0  cdwn: 4 
 22:25:04 | Ep: 401/ 600 | Trn loss:  0.229921 - Acc: 91.6335 | Val loss:  0.433649 - Acc: 87.3009 | last_lr: 1.56250e-05  bad_ep: 0  cdwn: 3                                                                 
 22:25:29 | Ep: 402/ 600 | Trn loss:  0.229780 - Acc: 91.6411 | Val loss:  0.434264 - Acc: 87.3380 | last_lr: 1.56250e-05  bad_ep: 0  cdwn: 2                                                                 
 22:25:54 | Ep: 403/ 600 | Trn loss:  0.229644 - Acc: 91.6432 | Val loss:  0.435028 - Acc: 87.3472 | last_lr: 1.56250e-05  bad_ep: 0  cdwn: 1                                                                 
 22:26:18 | Ep: 404/ 600 | Trn loss:  0.229513 - Acc: 91.6461 | Val loss:  0.435852 - Acc: 87.3611 | last_lr: 1.56250e-05  bad_ep: 0  cdwn: 0                                                                 
 22:26:43 | Ep: 405/ 600 | Tr

2024-09-30 23:05:51,923 - utils.utils_cellpainting - INFO: -  Model exported to NN_snnl_embd600_250Ltnt_512_20240917_2004_LAST_20240930_1930_ep_500.pt - epoch: 500


 23:05:51 | Ep: 500/ 600 | Trn loss:  0.223004 - Acc: 91.8669 | Val loss:  0.442907 - Acc: 87.3241 | last_lr: 7.81250e-06  bad_ep: 35  cdwn: 0 
 23:06:16 | Ep: 501/ 600 | Trn loss:  0.222962 - Acc: 91.8669 | Val loss:  0.442923 - Acc: 87.3194 | last_lr: 7.81250e-06  bad_ep: 36  cdwn: 0                                                                
 23:06:40 | Ep: 502/ 600 | Trn loss:  0.222921 - Acc: 91.8698 | Val loss:  0.442938 - Acc: 87.3148 | last_lr: 7.81250e-06  bad_ep: 37  cdwn: 0                                                                
 23:07:04 | Ep: 503/ 600 | Trn loss:  0.222879 - Acc: 91.8690 | Val loss:  0.442957 - Acc: 87.3102 | last_lr: 7.81250e-06  bad_ep: 38  cdwn: 0                                                                
 23:07:29 | Ep: 504/ 600 | Trn loss:  0.222838 - Acc: 91.8694 | Val loss:  0.442975 - Acc: 87.3102 | last_lr: 7.81250e-06  bad_ep: 39  cdwn: 0                                                                
 23:07:53 | Ep: 505/ 600 | T

2024-09-30 23:47:23,358 - utils.utils_cellpainting - INFO: -  Model exported to NN_snnl_embd600_250Ltnt_512_20240917_2004_LAST_20240930_1930_ep_600.pt - epoch: 600


 23:47:23 | Ep: 600/ 600 | Trn loss:  0.220171 - Acc: 91.9607 | Val loss:  0.441641 - Acc: 87.3519 | last_lr: 1.95313e-06  bad_ep: 13  cdwn: 0 


In [ ]:
# print(CKPT_FILE)
# save_checkpoint(end_epoch, model, optimizer, scheduler, metrics = metrics,
#                 filename = CKPT_FILE.format(ep=end_epoch),
#                 ckpt_path = CKPT_PATH, verbose = True)

In [ ]:
# start_epoch, end_epoch

# for mtrc in ['loss_trn', 'loss_val']:
#     for i in range(len(metrics[mtrc])):
#         # print(i)
#         metrics[mtrc][i] = metrics[mtrc][i].item()
 

# Modify TPSA Threshold

In [ ]:

train_y_72 = np.zeros_like(train_y)
train_y_72.shape[0]/3

In [ ]:
print(f"                  min           max           std          mean           median")
for x in ['TPSA', 'lnTPSA', 'log10TPSA']:
    print(f"{x:12s} {df_train[x].min():13.7f} {df_train[x].max():13.7f} {df_train[x].std():13.7f} {df_train[x].mean():13.7f} {df_train[x].median():13.7f}") 

df_train.TPSA.count()
df_train[df_train.TPSA >= THRESHOLD].TPSA.count()/df_train.TPSA.count()
df_train[df_train.TPSA < THRESHOLD].TPSA.count()/df_train.TPSA.count()


In [ ]:
_tmp = df_train.Metadata_Permiation.value_counts()
_tmp[0], _tmp[1]

In [ ]:
for threshold in [68, 69, 70, 71, 72, 100]:
    _tmp = (df_train['Metadata_TPSA'] >= threshold).value_counts()
    print(f"\n TPSA threshold {threshold} \n Total samples: {_tmp.sum()}")
    print(f" Label 0: {_tmp[False]:>7d}      % {_tmp[False]*100/_tmp.sum():2.2f} ")
    print(f" Label 1: {_tmp[True]:>7d}      % {_tmp[True]*100/_tmp.sum():2.2f} ")

In [ ]:
# fig, ax = plt.subplots(figsize=(4,4))
# fig.canvas.draw()  # Need to draw the figure to define renderer
# ax.set_title("AngleLabel example")
# # Plot two crossing lines and label each angle between them with the above
# center = (4.5, 650)
# p1 = [(2.5, 710), (6.0, 605)]
# p2 = [(3.0, 275), (5.5, 900)]
# line1, = ax.plot(*zip(*p1))
# line2, = ax.plot(*zip(*p2))
# point, = ax.plot(*center, marker="o")


In [ ]:
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
num_bins = 200
# fig, ax = plt.subplots()
fig = plt.figure(figsize=(10,5))
sigma = df_train.Metadata_TPSA.std()
mu = df_train.Metadata_TPSA.mean()
med = df_train.Metadata_TPSA.median()
# the histogram of the data
# We can set the number of bins with the *bins* keyword argument.
n, bins, patches = plt.hist(df_train.Metadata_TPSA, num_bins, density=False, range=[0, 500],)
# p1 = [(med, 710), (6.0, 605)]
# _ = plt.vlines(x=med, ymin=10, ymax=17000, colors='red', linestyles='-', lw=1.75, label='Single Short Line')
_ = plt.axvline(x=med, ymin=0, ymax=.97, color='red', linestyle='-', lw=1.75, label='Single Short Line')
_ = plt.xlabel('TPSA Value');
_ = plt.ylabel('# Compounds');
_ = plt.title(fr'TPSA distribution -  $\mu={mu:.3f}$    $\sigma={sigma:.3f}$')
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
# axs[1].hist(dist2, bins=n_bins)
plt.show()

# Stratified CV data splits

In [ ]:
def visualize_groups(classes, groups, name):
    # Visualize dataset groups
    fig, ax = plt.subplots(figsize=(10,5))
    ax.scatter(
        range(len(groups)),
        [0.5] * len(groups),
        c=groups,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.scatter(
        range(len(groups)),
        [3.5] * len(groups),
        c=classes,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.set(
        ylim=[-1, 5],
        yticks=[0.5, 3.5],
        yticklabels=["Data\ngroup", "Data\nclass"],
        xlabel="Sample index",
    )

In [ ]:
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    use_groups = "Group" in type(cv).__name__
    groups = group if use_groups else None
    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=groups)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["class", "group"]
    ax.set(
        yticks=np.arange(n_splits + 2) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
        xlim=[0, 100],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch


In [ ]:
rng = np.random.RandomState(1338)
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
n_splits = 4

# Generate the class/group data
# n_points = 100
# X = rng.randn(100, 10)

# percentiles_classes = [0.1, 0.3, 0.6]
# y = np.hstack([[ii] * int(100 * perc) for ii, perc in enumerate(percentiles_classes)])

In [ ]:
# Generate uneven groups

# group_prior = rng.dirichlet([2] * 10)
# group_prior.sum()
# group_prior

# groups = np.repeat(np.arange(10), rng.multinomial(100, group_prior))
# groups.shape
# groups

In [ ]:
groups = np.repeat(0, train_X.shape[0])
groups.shape

In [ ]:
visualize_groups(train_y, groups, "no groups")

In [ ]:
n_splits = 5
groups = None

In [ ]:
fig, ax = plt.subplots()
cv = KFold(n_splits)
plot_cv_indices(cv, train_X, train_y, groups, ax, n_splits)

# Input 

## Read Embedded Features CSV file

In [ ]:
BASE_TRAIN_INPUT = os.path.join(OUTPUT_PATH, INPUT_FILE.format(runmode = BASE_runmode ,datatype='train'))
BASE_TEST_INPUT  = os.path.join(OUTPUT_PATH, INPUT_FILE.format(runmode = BASE_runmode ,datatype='test'))
SNNL_TRAIN_INPUT = os.path.join(OUTPUT_PATH, INPUT_FILE.format(runmode = SNNL_runmode ,datatype='train'))
SNNL_TEST_INPUT  = os.path.join(OUTPUT_PATH, INPUT_FILE.format(runmode = SNNL_runmode ,datatype='test'))
BASE_TRAIN_INPUT
BASE_TEST_INPUT 
SNNL_TRAIN_INPUT
SNNL_TEST_INPUT 

In [ ]:
df_test = pd.read_csv(SNNL_TEST_INPUT )
df_train = pd.read_csv(SNNL_TRAIN_INPUT)

# df_train = pd.read_csv(BASE_TRAIN_INPUT)
# df_test = pd.read_csv(BASE_TEST_INPUT )
# df_train = pd.read_csv(TRAIN_INPUT, nrows = 100 )
# df_train = pd.read_csv(TRAIN_INPUT, usecols = ['Metadata_Batch'])

In [ ]:
df_test.info()
df_test.shape
df_test.columns
df_test.iloc[:5,:13]

In [ ]:
# df_train = pd.read_csv(TRAIN_INPUT, nrows = 100 )
# df_train = pd.read_csv(TRAIN_INPUT, usecols = ['Metadata_Batch'])

In [ ]:
test_y = df_test.iloc[:,10:11].to_numpy().ravel().astype(np.uint8)
test_y.sum()
test_y.shape, type(test_y), test_y.dtype
test_X = df_test.iloc[:,11:].to_numpy()
test_X.shape,type(test_X), test_X.dtype

In [ ]:
df_train.shape
df_train.info()
df_train.iloc[:5,:16]

In [ ]:
312000+34542

In [ ]:
train_y = df_train.iloc[:,10:11].to_numpy().ravel().astype(np.uint8)
train_y.sum()
train_y.shape, type(train_y), train_y.dtype

train_X = df_train.iloc[:,11:].to_numpy()
train_X.shape,type(train_X) ,train_X.dtype

## Standardize inputs

In [ ]:
print(f"Train_X :  Min: {train_X.min():.4f}    Max: {train_X.max():.4f}   Mean: {train_X.mean():.4f}  Std: {train_X.std():.4f}")
print(f"Test_X  :  Min: {test_X.min():.4f}    Max: {test_X.max():.4f}    Mean: {test_X.mean():.4f}  Std: {test_X.std():.4f}")

In [ ]:
# print(f"Train_X :  Min: {train_X.min():.4f}    Max: {train_X.max():.4f}   Mean: {train_X.mean():.4f}  Std: {train_X.std():.4f}")
# print(f"Test_X  :  Min: {test_X.min():.4f}    Max: {test_X.max():.4f}    Mean: {test_X.mean():.4f}  Std: {test_X.std():.4f}")

In [ ]:
scaler = StandardScaler(copy = True)
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [ ]:
print("After Standard Scaler Transformation")
print(f"Train_X :  Min: {train_X.min():.4f}    Max: {train_X.max():.4f}   Mean: {train_X.mean():.4f}  Std: {train_X.std():.4f}")
print(f"Test_X  :  Min: {test_X.min():.4f}    Max: {test_X.max():.4f}    Mean: {test_X.mean():.4f}  Std: {test_X.std():.4f}")


In [ ]:
label_counts([("Training", train_y), ("Test", test_y)])

# TQDM Examples

In [ ]:
#-----------------------------------------
# TRANGE example
#-----------------------------------------   
# with trange(+1, ns.trn_iters_warmup+1 , initial = 0 , total = ns.trn_iters_warmup, position=0, file=sys.stdout,
#             leave= False, disable = disable_tqdm, desc=f" Warmup Epoch {ns.current_epoch}/{ns.stop_epoch_warmup}") as t_warmup :
#     for _ in t_warmup:
#         ns.current_iter += 1            

#         batch = next(dldrs.warmup_trn_loader)            
#         environ.set_inputs(batch, input_size)

#         environ.optimize(is_policy=False, 
#                          num_train_layers=ns.num_train_layers,
#                          flag='update_weights', 
#                          verbose = verbose)
    
#         t_warmup.set_postfix({'curr_iter':ns.current_iter, 
#                             'Loss': f"{environ.losses['total']['total'].item():.4f}"})

#-----------------------------------------
# TQDM example
#-----------------------------------------   
# current_epoch = 1
# total_epochs = 20
# current_iter = 0 
# train_minibatches = len(data_loader['train']) // minibatch_size
# val_minibatches = len(data_loader['val']) // minibatch_size

# # with batch_count, (batch_features, batch_labels, _, _, _, _) in tqdm(enumerate(data_loader['train']), initial=0, total = 400, position=0, file=sys.stdout,
#             # leave= False, desc=f" Epoch {current_epoch}/{total_epochs}") as t_warmup :
# t_warmup =  tqdm(enumerate(data_loader['train']), initial=0, total = train_minibatches, position=0, file=sys.stdout,
#             leave= False, desc=f" Epoch {current_epoch}/{total_epochs}") 
# for batch_count, (batch_features, batch_labels, _, _, _, _) in t_warmup:
#     # batch_count, (batch_features, batch_labels, _, _, _, _) = pp
#     loss = random.random()
#     t_warmup.set_postfix({'curr_iter':batch_count, 'Loss': f"{loss:.4f}"})